# THIS CODE CALCULATE THE MONTHLY MEAN OF CO and O3

## A. Read Myanmar coordinates

In [3]:
import pandas as pd 
import numpy as np
import geopandas as gpd

In [4]:
# read lat and lon data
coord = pd.read_csv('myanmar_coords.csv')
# a function to trucate the lat and lon to 4 decimal places, keep zeros


def truncate(num):
    return [round(x, 4) for x in num]


# create a tuple of (Name, lat, lon)
coord['coordinates'] = list(
    zip(
        coord['Name'],
        truncate(coord['Latitude']),
        truncate(coord['Longitude'])
    ))

print(coord['coordinates'])

0       (Ayeyarwady, 16.834, 95.1805)
1            (Bago, 18.2457, 96.1005)
2            (Chin, 22.0182, 93.5038)
3          (Sagaing, 24.4284, 95.394)
4          (Kachin, 26.0208, 97.4919)
5           (Kayah, 19.2813, 97.3292)
6           (Kayin, 17.1863, 97.7499)
7          (Magway, 20.2748, 94.7362)
8        (Mandalay, 20.9872, 95.7657)
9       (Naypyitaw, 20.2807, 96.2659)
10        (Rakhine, 19.5213, 94.0072)
11           (Shan, 21.5122, 98.0098)
12    (Tanintharyi, 13.0143, 98.7442)
13         (Yangon, 16.6476, 96.1129)
14            (Mon, 16.3003, 97.6982)
Name: coordinates, dtype: object


Create a dataframe based on the coordinates

In [5]:
# make a new dataframe from the coordinates
df = pd.DataFrame(coord['coordinates'].tolist(), columns=['NAME_1', 'lat', 'lon'])
df

,NAME_1,lat,lon
0,Ayeyarwady,16.8340,95.1805
1,Bago,18.2457,96.1005
2,Chin,22.0182,93.5038
3,Sagaing,24.4284,95.3940
4,Kachin,26.0208,97.4919
5,Kayah,19.2813,97.3292
6,Kayin,17.1863,97.7499
7,Magway,20.2748,94.7362
8,Mandalay,20.9872,95.7657
9,Naypyitaw,20.2807,96.2659


When you donwload the data from NASA, you need to know the max and min of lat and lon. Make a function to calculate those values

In [6]:
lat_max = df['lat'].max()
lat_min = df['lat'].min()
lon_max = df['lon'].max()
lon_min = df['lon'].min()

## B. Downloading Data based on NASA guidelines

Link: https://disc.gsfc.nasa.gov/information/howto?keywords=API&title=How%20to%20Use%20the%20Web%20Services%20API%20for%20Subsetting%20MERRA-2%20Data

### 1) Example #1: MERRA-2 3-Hourly 3-Dimensional Instantaneous Variables (M2I3NPASM) 

The data product used in this first example is the MERRA-2 product M2I3NPASM which contains instantaneous values at 3-hour intervals of 3-dimensional meteorological fields on pressure levels. Individual data granules for this product are larger than 1 GB and contain data for one day with eight time steps per file. The example shown here will perform subsetting by: variable, spatial domain, vertical dimension, and time of day. This reduces the data volume to 0.3% of it's original size.

The first step is to import the required Python libraries. If any of the following import commands fail check the local Python environment and install any missing packages.

In [6]:
# STEP 1
import sys
import json
import urllib3
import certifi
import requests
from time import sleep
from http.cookiejar import CookieJar
import urllib.request
from urllib.parse import urlencode
import getpass

The second step is to initialize the urllib PoolManager and set the base URL for the API requests that will be sent to the GES DISC subsetting service.

In [7]:
# STEP 2
# Create a urllib PoolManager instance to make requests.
http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED',ca_certs=certifi.where())
# Set the URL for the GES DISC subset service endpoint
url = 'https://disc.gsfc.nasa.gov/service/subset/jsonwsp'

The third step defines a local general-purpose method that submits a JSON-formatted Web Services Protocol (WSP) request to the GES DISC server, checks for any errors, and then returns the response. This method is created for convenience as this task will be repeated more than once.

In [8]:
# STEP 3
# This method POSTs formatted JSON WSP requests to the GES DISC endpoint URL
# It is created for convenience since this task will be repeated more than once
def get_http_data(request):
    hdrs = {'Content-Type': 'application/json',
            'Accept'      : 'application/json'}
    data = json.dumps(request)       
    r = http.request('POST', url, body=data, headers=hdrs)
    response = json.loads(r.data)   
    # Check for errors
    if response['type'] == 'jsonwsp/fault' :
        print('API Error: faulty %s request' % response['methodname'])
        sys.exit(1)
    return response

The fourth step defines the specific details of the subset. In this example three variables are selected: air temperature (T), relative humidity (RH), and ozone mass mixing ratio (O3). The desired spatial coverage is in the Southern Hemisphere poleward of 45S. The date range for our example is 1-5 January 1980 and we will extract data samples only at 00Z. The vertical dimension is restricted to just the mandatory pressure levels. These details are coded as local variables so they can be easily changed for different use cases.

You can use the API to do a [Dataset Search](https://disc.gsfc.nasa.gov/information/howto?keywords=api&title=How%20to%20Use%20the%20Web%20Services%20API%20for%20Dataset%20Searching) in order to find out the exact name of the data product and its variables. Alternatively, a single data granule can be downloaded to find out the variable names. Note that variable names are case sensitive.

In [9]:
# STEP 4
# Define the parameters for the data subset
product = 'M2I3NVCHM_V5.12.4' 
varNames =['CO', 'O3']
minlon = lon_min
maxlon = lon_max
minlat = lat_min
maxlat = lat_max
begTime = '2015-01-01'
endTime = '2015-12-31'
begHour = '00:00'
endHour = '00:00'


# Subset only the mandatory pressure levels (units are hPa)
# 1000 925 850 700 500 400 300 250 200 150 100 70 50 30 20 10 7 5 3 2 1 
dimName = 'lev'
dimVals = [1,4,7,13,17,19,21,22,23,24,25,26,27,29,30,31,32,33,35,36,37]
# Construct the list of dimension name:value pairs to specify the desired subset
dimSlice = []
for i in range(len(dimVals)):
    dimSlice.append({'dimensionId': dimName, 'dimensionValue': dimVals[i]})


In the fifth step, the desired spatial and temporal constraints, along with the dataset and variable specifications, are stored in a JSON-based Web Service Protocol (WSP) structure, which is named “subset_request”.

In [10]:
# STEP 5
# Construct JSON WSP request for API method: subset

subset_request = {
    'methodname': 'subset',
    'type': 'jsonwsp/request',
    'version': '1.0',
    'args': {
        'role'  : 'subset',
        'start' : begTime,
        'end'   : endTime,
        'box'   : [minlon, minlat, maxlon, maxlat],
        'crop'  : True, 
        'data': [{'datasetId': product,
                  'variable' : varNames[0],
                  'slice': dimSlice
                 },
                 {'datasetId': product,
                  'variable' : varNames[1],
                  'slice'    : dimSlice
                 }
        ]
    }
}

**Note:** The two code blocks above (where the subset parameters are defined and the subset_request structure is assembled) are the only sections of this example that need to be modified for other use cases. Please scroll down to the bottom of this page to see the second example that illustrates how to create a daily average of 2-dimensional hourly variables.

The top level parameters in the subset_request structure shown above are: methodname, type, version, and args. The args contain parameters to customize the subset, which in this case are: role, start, end, box, crop, and data.

For API subset requests the parameters methodname and role should always be set to 'subset'. The remaining parameter values are set using the local variables we defined in the previous code block. The start and end parameters contain the date range. The box parameter specifies the desired spatial domain that will constrain the granule search — only data granules within the spatial domain will be returned. The crop parameter is a True/False flag that indicates whether to perform spatial subsetting on the granules returned by the spatial search. Granules will not be trimmed to the specified spatial domain unless crop is set to True.

The data parameter section includes three sub-parameters: datasetId, variable, and slice. If you wish to retrieve all vertical dimensions, omit the slice parameter; if you wish to retrieve all the variables in the data file, omit the variable parameter. Please refer to the [Complete reference documentation for the GES DISC Subsetting Service API](https://disc.gsfc.nasa.gov/service/subset) for more information.

In the sixth step, the JSON-formatted subset_request is posted to the GES DISC server. The Job ID is extracted from the response and will be used later as a reference for the request.

In [ ]:
# STEP 6
# Submit the subset request to the GES DISC Server
response = get_http_data(subset_request)
# Report the JobID and initial status
myJobId = response['result']['jobId']
print('Job ID: '+myJobId)
print('Job status: '+response['result']['Status'])

At this point the job is running on the GES DISC server. The seventh step is to construct another JSON WSP status_request, with methodname parameter set to 'GetStatus'. The args parameter contains the extracted Job ID. The status_request is submitted periodically to monitor the job status as it changes from 'Accepted' to 'Running' to '100% completed'. When the job is finished check on the final status to ensure the job succeeded.

In [ ]:
# STEP 7
# Construct JSON WSP request for API method: GetStatus
status_request = {
    'methodname': 'GetStatus',
    'version': '1.0',
    'type': 'jsonwsp/request',
    'args': {'jobId': myJobId}
}

# Check on the job status after a brief nap
while response['result']['Status'] in ['Accepted', 'Running']:
    sleep(5)
    response = get_http_data(status_request)
    status  = response['result']['Status']
    percent = response['result']['PercentCompleted']
    print ('Job status: %s (%d%c complete)' % (status,percent,'%'))
if response['result']['Status'] == 'Succeeded' :
    print ('Job Finished:  %s' % response['result']['message'])
else : 
    print('Job Failed: %s' % response['fault']['code'])
    sys.exit(1)

After confirming that the job has finished successfully it is time to retrieve the results. The results of a subset request job are URLs: there are HTTP_Services URLs (one for every data granule in the time range of interest) plus links to any relevant documentation. Each HTTP_Services URL contains the specifics of the subset request encoded as facets. Data subsets and documentation files are downloaded using the requests Python library.

There are two ways to retrieve the list of URLs when the subset job is finished:

**Method 1:** Use the API method named GetResult. This method will return the URLs along with three additional attributes: a label, plus the beginning and ending time stamps for that particular data granule. The label serves as the filename for the downloaded subsets.

**Method 2:** Retrieve a plain-text list of URLs in a single shot using the saved JobID. This is a shortcut to retrieve just the list of URLs without any of the other metadata.

The next step for **Method 1** is to construct a third type of JSON WSP request that retrieves the results of this Job. When that request is submitted the results are returned in batches of 20 items, starting with item 0. The startIndex value in the results_request structure must be updated after each successive batch is retrieved.

In [ ]:
# STEP 8 (Plan A - preferred)
# Construct JSON WSP request for API method: GetResult
batchsize = 20
results_request = {
    'methodname': 'GetResult',
    'version': '1.0',
    'type': 'jsonwsp/request',
    'args': {
        'jobId': myJobId,
        'count': batchsize,
        'startIndex': 0
    }
}

# Retrieve the results in JSON in multiple batches 
# Initialize variables, then submit the first GetResults request
# Add the results from this batch to the list and increment the count
results = []
count = 0 
response = get_http_data(results_request) 
count = count + response['result']['itemsPerPage']
results.extend(response['result']['items']) 

# Increment the startIndex and keep asking for more results until we have them all
total = response['result']['totalResults']
while count < total :
    results_request['args']['startIndex'] += batchsize 
    response = get_http_data(results_request) 
    count = count + response['result']['itemsPerPage']
    results.extend(response['result']['items'])
       
# Check on the bookkeeping
print('Retrieved %d out of %d expected items' % (len(results), total))

Below is the code for **Method 2**. Construct a request using the saved JobID and retrieve the results with the requests library. If the requests.get() method does not return an error, the URLs are stored locally and printed out for informational purposes.

In [ ]:
# STEP 8 (Plan B)
# Retrieve a plain-text list of results in a single shot using the saved JobID

result = requests.get('https://disc.gsfc.nasa.gov/api/jobs/results/'+myJobId)
try:
    result.raise_for_status()
    urls = result.text.split('\n')
    for i in urls : print('\n%s' % i)
except :
    print('Request returned error code %d' % result.status_code)


It is important to keep in mind that the results returned at this point are not data files but lists of URLs. Most of the URLs will contain HTTP_Services requests to actually do the subsetting and return the data, but some of them may be links to documentation files pertaining to the dataset in question.

It is worthwhile to separate the document URLs from the HTTP_services URLs in case the documentation has already been retrieved so they won't be downloaded again. The way we do this is to check for start and end attributes which are always associated with HTTP_services URLs. The remainder of the example code assumes the use of **Method 1** because it makes use of this extra metadata.

In [ ]:
# Sort the results into documents and URLs

docs = []
urls = []
for item in results :
    try:
        if item['start'] and item['end'] : urls.append(item) 
    except:
        docs.append(item)
# Print out the documentation links, but do not download them
# print('\nDocumentation:')
# for item in docs : print(item['label']+': '+item['link'])

The final step is to invoke each HTTP_Services URL and download the data files. The contents of the label attribute are used here as the output file name, but the name can be any string. It is important to download each file one at a time, in series rather than in parallel, to avoid overloading the GES DISC servers.

We show two methods for downloading the data files using either the requests.get() or the urllib.request() modules. Use one or the other, but not both. If the requests.get() method fails try the alternate code block, but be sure to update it with a proper Earthdata login name and password.

**Download with Requests Library:**  
In STEP 10, for the request.get() module to work properly, you must have a [HOME/.netrc](https://disc.gsfc.nasa.gov/data-access) file that contains the following text (configured with your own Earthdata userid and password): machine urs.earthdata.nasa.gov login [userid] password [password]. In the alternate STEP 10, you can provide your userid and password on-the-fly when running the code.

In [ ]:
# STEP 10 
# Use the requests library to submit the HTTP_Services URLs and write out the results.
print('\nHTTP_services output:')
for item in urls :
    URL = item['link'] 
    result = requests.get(URL)
    try:
        result.raise_for_status()
        outfn = item['label']
        f = open(outfn,'wb')
        f.write(result.content)
        f.close()
        print(outfn, "is downloaded")
    except:
        print('Error! Status code is %d for this URL:\n%s' % (result.status.code,URL))
        print('Help for downloading data is at https://disc.gsfc.nasa.gov/data-access')
        
print('Downloading is done and find the downloaded files in your current working directory')

## C. CLEANING THE DATA

Following these steps first:
... in the same directory as this notebook,
1. Copy all nc files to a new folder and name it as 'CO_O3'
2. Make a new folder and name it 'CO_O3_CSV' to store the csv results files

In [ ]:
# READ THE PATH OF THE DOWNLOADED FILES
import os 
full_path = os.getcwd() + '/' + 'CO_O3'
full_path

In [2]:
# read all the nc files in the directory
import xarray as xr
ds = xr.open_mfdataset(full_path + '/*.nc', combine='by_coords')
ds

<xarray.Dataset>
Dimensions:  (time: 14776, lon: 8, lat: 26, lev: 1)
Coordinates:
  * time     (time) datetime64[ns] 2011-01-01 ... 2016-12-31T21:00:00
  * lon      (lon) float64 93.75 94.38 95.0 95.62 96.25 96.88 97.5 98.12
  * lat      (lat) float64 13.5 14.0 14.5 15.0 15.5 ... 24.0 24.5 25.0 25.5 26.0
  * lev      (lev) float64 1.0
Data variables:
    CO       (time, lev, lat, lon) float32 dask.array<chunksize=(1, 1, 26, 8), meta=np.ndarray>
    O3       (time, lev, lat, lon) float32 dask.array<chunksize=(1, 1, 26, 8), meta=np.ndarray>
Attributes: (12/32)
    CDI:                               Climate Data Interface version 1.9.8 (...
    Conventions:                       CF-1
    History:                           Original file generated: Thu Jun 12 19...
    Comment:                           GMAO filename: d5124_m2_jan10.inst3_3d...
    Filename:                          MERRA2_400.inst3_3d_chm_Nv.20110101.nc4
    Institution:                       NASA Global Modeling and Assimilation ...
    ...                                ...
    RangeBeginningDate:                2011-01-01
    RangeBeginningTime:                00:00:00.000000
    RangeEndingDate:                   2011-01-01
    RangeEndingTime:                   21:00:00.000000
    history_L34RS:                     'Created by L34RS v1.4.3 @ NASA GES DI...
    CDO:                               Climate Data Operators version 1.9.8 (...

In [8]:
# TESTING
# extract the data of CO and O3 with 'lev'==1 with lat and lon closest to the coordinates
# create a new dataframe to store the data

lat = df['lat'][0]
lon = df['lon'][0]
NAME_1 = df['NAME_1'][0]
lev = 1
# select co and o3 data with the closest lat and lon and the time
co = ds['CO'].sel(lat=lat, lon=lon, lev=lev, method='nearest')
o3 = ds['O3'].sel(lat=lat, lon=lon, lev=lev, method='nearest')
time = ds['time']


# create a new dataframe to store the data
df2 = pd.DataFrame({'CO': co.values, 'O3': o3.values, 'time': time.values})


# convert the time to datetime
df2['time'] = pd.to_datetime(df2['time'])

# group the data by month for each year, and calculate the mean of CO and O3, make a new dataframe
df2['year'] = df2['time'].dt.year
df2['month'] = df2['time'].dt.month

df3 = df2.groupby(['year', 'month']).mean().reset_index()

#make NAME_1 and lat and lon as a list
NAMES_1 = []
lats = []
lons = []
for i in range(len(df3['time'])):
    NAMES_1.append(NAME_1)
    lats.append(lat)
    lons.append(lon)

df3['NAME_1'] = NAMES_1
df3['lat'] = lats
df3['lon'] = lons

df3

,CO,O3,time
0,3.535058e-09,1.684765e-05,2011-01-01 00:00:00
1,3.514226e-09,1.103814e-07,2011-01-01 03:00:00
2,3.535224e-09,1.115017e-07,2011-01-01 06:00:00
3,3.571672e-09,1.067598e-07,2011-01-01 09:00:00
4,3.614422e-09,1.628135e-05,2011-01-01 12:00:00
...,...,...,...
14771,3.987696e-09,1.085480e-07,2016-12-31 09:00:00
14772,3.889262e-09,1.655708e-05,2016-12-31 12:00:00
14773,3.837722e-09,1.651230e-05,2016-12-31 15:00:00
14774,3.860142e-09,1.658693e-05,2016-12-31 18:00:00


Below is a function to extract the data from the nc files based on the lat, lon provided. The default value of levitation is 1.

In [11]:
def get_data(lat, lon, NAME_1, ds, lev=1):
    # select CO, O3, and time from the data
    co = ds['CO'].sel(lat=lat, lon=lon, lev=lev, method='nearest')
    o3 = ds['O3'].sel(lat=lat, lon=lon, lev=lev, method='nearest')
    time = ds['time']
    # create a new dataframe to store the data
    df2 = pd.DataFrame({'CO': co.values, 'O3': o3.values, 'time': time.values})
    df2['time'] = pd.to_datetime(df2['time'])
    df2['month'] = df2['time'].dt.month
    df2['year'] = df2['time'].dt.year
    # group the data by month, and calculate the mean of CO and O3, make a new dataframe
    df3 = df2.groupby(['year', 'month']).mean()
    df3 = df3.reset_index()
    #make NAME_1 and lat and lon as a list to add back to the dataframe
    NAMES_1 = []
    lats = []
    lons = []
    for i in range(len(df3['time'])):
        NAMES_1.append(NAME_1)
        lats.append(lat)
        lons.append(lon)
    # add the NAME_1, lat, and lon to the dataframe
    df3['NAME_1'] = NAMES_1
    df3['lat'] = lats
    df3['lon'] = lons
    return df3

In [14]:
# LOOP THROUGH THE MYANMAR LAT-LON COORDINATES TO GET THE DATA
for i in range(len(df['NAME_1'])):
    NAME_1 = df['NAME_1'][i]
    lat = df['lat'][i]
    lon = df['lon'][i]
    df2 = get_data(lat, lon, NAME_1, ds)
    # export the data to a csv file
    df2.to_csv('CO_O3_csv/{}_CO_O3.csv'.format(NAME_1), index=False)
print('All the data are exported to csv files')